In [1]:
from transformers import LlamaForCausalLM, LlamaTokenizer
import os
import numpy as np
import torch

In [2]:
base_model = "meta-llama/Llama-2-7b-chat-hf"
device_map = 'auto'
max_memory_mapping = {0: "23GiB", 1: "23GiB"}

In [3]:
tokenizer = LlamaTokenizer.from_pretrained(base_model,
                                           padding_side = "left",
                                           add_eos_token = True,
                                           add_bos_token = True,                                        
                                           token = os.environ.get("HUGGINGFACE_ACCESS_TOKEN"),
)
tokenizer.add_special_tokens({"pad_token":"<pad>"})

1

In [4]:
print(f"pad_token_id={tokenizer.pad_token_id}")

pad_token_id=32000


In [5]:
model = LlamaForCausalLM.from_pretrained(
        base_model,
        load_in_8bit=True,
        torch_dtype=torch.float16,
        device_map=device_map,
        max_memory = max_memory_mapping,
        token = os.environ.get("HUGGINGFACE_ACCESS_TOKEN"),
    ).eval()
model.resize_token_embeddings(len(tokenizer),pad_to_multiple_of=8)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Embedding(32008, 4096)

In [6]:
tokenizer.vocab_size

32000

In [8]:
prompt = ["Hey, are you conscious? Can you talk to me?",
          "Once upon a time there was a ghost ",
          """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n### Instruction:\nGiven the user preference and unpreference, identify whether the user will like the target movie by answering "Yes." or "No.".\n\n### Input:\n\nUser Preference: "Twister (1996)", "Con Air (1997)", "River Wild, The (1994)", "13th Warrior, The (1999)", "Batman Returns (1992)", "From Dusk Till Dawn (1996)", "Young Guns II (1990)", "Demolition Man (1993)", "Mummy, The (1999)", "Eraser (1996)", "Substitute, The (1996)", "Maximum Risk (1996)", "Sudden Death (1995)", "Outbreak (1995)", "Dick Tracy (1990)",\nUser Unpreference:"Cliffhanger (1993)","Armageddon (1998)","Mercury Rising (1998)","U.S. Marshalls (1998)","Shadow, The (1994)",\n Whether the user will like the target movie "Rocketeer, The (1991)\\?\n\n### Response:\nNo.</s>"""
          ]
inputs = tokenizer(prompt, return_tensors="pt", padding = True)

In [9]:
inputs

{'input_ids': tensor([[32000, 32000, 32000,  ...,   592, 29973,     2],
        [32000, 32000, 32000,  ...,  3069, 29871,     2],
        [    1, 13866,   338,  ..., 29889,     2,     2]]), 'attention_mask': tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])}

In [12]:
generate_ids = model.generate(inputs.input_ids, max_length=256)
print(generate_ids)

/home/grads/m/mbismay/miniconda3/envs/mixtral/lib/python3.9/site-packages/transformers/generation/utils.py:1141: UserWarning: Input length of input_ids is 341, but `max_length` is set to 256. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
/home/grads/m/mbismay/miniconda3/envs/mixtral/lib/python3.9/site-packages/transformers/generation/utils.py:1413: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


tensor([[32000, 32000, 32000,  ..., 29973,     2,     1],
        [32000, 32000, 32000,  ..., 29871,     2,     1],
        [    1, 13866,   338,  ...,     2,     2,    13]])


In [13]:
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)

['Hey, are you conscious? Can you talk to me?',
 'Once upon a time there was a ghost ',
 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n### Instruction:\nGiven the user preference and unpreference, identify whether the user will like the target movie by answering "Yes." or "No.".\n\n### Input:\n\nUser Preference: "Twister (1996)", "Con Air (1997)", "River Wild, The (1994)", "13th Warrior, The (1999)", "Batman Returns (1992)", "From Dusk Till Dawn (1996)", "Young Guns II (1990)", "Demolition Man (1993)", "Mummy, The (1999)", "Eraser (1996)", "Substitute, The (1996)", "Maximum Risk (1996)", "Sudden Death (1995)", "Outbreak (1995)", "Dick Tracy (1990)",\nUser Unpreference:"Cliffhanger (1993)","Armageddon (1998)","Mercury Rising (1998)","U.S. Marshalls (1998)","Shadow, The (1994)",\n Whether the user will like the target movie "Rocketeer, The (1991)\\?\n\n### Response:\nNo.\n']

In [14]:
tokenizer.eos_token

'</s>'

In [16]:
prompt = ["[INST]"]
inputs = tokenizer(prompt, return_tensors="pt", padding = True)
print(inputs)

{'input_ids': tensor([[    1,   518, 25580, 29962,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}


In [46]:
tokenizer.batch_decode([2], skip_special_tokens=True, clean_up_tokenization_spaces=True)

['</s>']

In [44]:
l = [[1,2,1,3,1,3,4,45,1,3], [1,2,1,3,1,3,4,45,1,3]]
l1 = [[n if n!=1 else 0 for n in label] for label in l]
l1, l

([[0, 2, 0, 3, 0, 3, 4, 45, 0, 3], [0, 2, 0, 3, 0, 3, 4, 45, 0, 3]],
 [[1, 2, 1, 3, 1, 3, 4, 45, 1, 3], [1, 2, 1, 3, 1, 3, 4, 45, 1, 3]])